In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import sys
import re



In [2]:
skillType=pd.read_csv("vizwiz/vizwiz_skill_typ_train.csv",engine="python")

skillType.head(3)

,qid,IMG,QSN,ANS_TYP,TXT,OBJ,COL,CNT,OTH,ANS1,ANS2,ANS3,ANS4,ANS5,ANS6,ANS7,ANS8,ANS9,ANS10
0,VizWiz_train_000000000000.jpg,VizWiz_train_000000000000.jpg,What's the name of this product?,other,1,1,0,0,0,basil leaves,basil leaves,basil,basil,basil leaves,basil leaves,basil leaves,basil leaves,basil leaves,basil
1,VizWiz_train_000000000001.jpg,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,other,1,1,0,0,0,soda,coca cola,coca cola,unsuitable,unsuitable,coke 0,coca cola 0,coke 0,coca cola,coke
2,VizWiz_train_000000000002.jpg,VizWiz_train_000000000002.jpg,Is this enchilada sauce or is this tomatoes? ...,other,1,1,0,0,0,these tomatoes not enchilada sauce,tomatoes,tomatoes,tomatoes,tomatoes,crushed tomatoes,crushed tomatoes,tomatoes,tomatoes,tomatoes


In [3]:
#color recognition
colorRecognition=pd.read_csv("vizwiz/vizwiz_train_color_recognition.csv", delimiter=";", engine="python",error_bad_lines=False)
colorRecognition.head(3)


Skipping line 4546: Expected 5 fields in line 4546, saw 6
Skipping line 4816: Expected 5 fields in line 4816, saw 6


,qid,question,descriptions,tags,dominant_colors
0,VizWiz_train_000000000007.jpg,This item.,"[{'confidence': 0.3326841203874476, 'text': 'a...","['person', 'indoor', 'hand', 'small', 'sitting...",['Black']
1,VizWiz_train_000000000008.jpg,What color do these look?,[],"['indoor', 'table', 'food', 'sitting', 'small'...","['Black', 'Pink']"
2,VizWiz_train_000000000010.jpg,Is this.,"[{'confidence': 0.9337091944456669, 'text': 'a...","['person', 'indoor', 'electronics', 'cellphone...","['Black', 'White']"


In [4]:
colorRecognition['answer']=colorRecognition['descriptions']

colorRecognition.head(3)

,qid,question,descriptions,tags,dominant_colors,answer
0,VizWiz_train_000000000007.jpg,This item.,"[{'confidence': 0.3326841203874476, 'text': 'a...","['person', 'indoor', 'hand', 'small', 'sitting...",['Black'],"[{'confidence': 0.3326841203874476, 'text': 'a..."
1,VizWiz_train_000000000008.jpg,What color do these look?,[],"['indoor', 'table', 'food', 'sitting', 'small'...","['Black', 'Pink']",[]
2,VizWiz_train_000000000010.jpg,Is this.,"[{'confidence': 0.9337091944456669, 'text': 'a...","['person', 'indoor', 'electronics', 'cellphone...","['Black', 'White']","[{'confidence': 0.9337091944456669, 'text': 'a..."


In [5]:
colorResult = pd.merge(colorRecognition, skillType, how='left', on='qid')
colorResult

,qid,question,descriptions,tags,dominant_colors,answer,IMG,QSN,ANS_TYP,TXT,...,ANS1,ANS2,ANS3,ANS4,ANS5,ANS6,ANS7,ANS8,ANS9,ANS10
0,VizWiz_train_000000000007.jpg,This item.,"[{'confidence': 0.3326841203874476, 'text': 'a...","['person', 'indoor', 'hand', 'small', 'sitting...",['Black'],"[{'confidence': 0.3326841203874476, 'text': 'a...",VizWiz_train_000000000007.jpg,This item.,other,1,...,monitor cleaning kit,monitor cleaning kit,cleaning kit,monitor cleaning kit,monitor cleaning kit,monitor cleaner,monitor cleaning kit,monitor cleaning kit,monitor cleaning kit,monitor cleaning kit
1,VizWiz_train_000000000008.jpg,What color do these look?,[],"['indoor', 'table', 'food', 'sitting', 'small'...","['Black', 'Pink']",[],VizWiz_train_000000000008.jpg,What color do these look?,other,0,...,orange,orange,orange,pink,orange,orange,pink,orange,orange,orange
2,VizWiz_train_000000000010.jpg,Is this.,"[{'confidence': 0.9337091944456669, 'text': 'a...","['person', 'indoor', 'electronics', 'cellphone...","['Black', 'White']","[{'confidence': 0.9337091944456669, 'text': 'a...",VizWiz_train_000000000010.jpg,Is this.,other,0,...,android phone,cell phone,cell phone,cellphone,samsung phone,unanswerable,phone,smart phone,white smartphone,phone
3,VizWiz_train_000000000012.jpg,what is this?,"[{'confidence': 0.910822291974947, 'text': 'a ...","['person', 'electronics', 'remote', 'calculato...","['Black', 'Grey', 'White']","[{'confidence': 0.910822291974947, 'text': 'a ...",VizWiz_train_000000000012.jpg,what is this?,other,0,...,remote,remote control,jvc dvd vcr remote,remote,remote,remote control,remote,remote,tv remote,remote
4,VizWiz_train_000000000013.jpg,What kind of drink is this?,"[{'confidence': 0.5964878314966942, 'text': 'a...","['indoor', 'bed', 'laying', 'sitting', 'table'...","['Grey', 'White', 'Brown']","[{'confidence': 0.5964878314966942, 'text': 'a...",VizWiz_train_000000000013.jpg,What kind of drink is this?,other,1,...,soda,dr pepper,dr pepper,soda,dr pepper,dr pepper,energy drink,dr pepper,dr pepper,dr pepper
5,VizWiz_train_000000000015.jpg,What color... what color is this skirt?,"[{'confidence': 0.3852366433305019, 'text': 'a...","['indoor', 'food', 'sitting', 'table', 'white'...","['Grey', 'Brown']","[{'confidence': 0.3852366433305019, 'text': 'a...",VizWiz_train_000000000015.jpg,What color... what color is this skirt?,other,0,...,white,grey,white,white,white,white,white,white,white,white
6,VizWiz_train_000000000019.jpg,Can I ask about anything or any information wi...,"[{'confidence': 0.4687312594679555, 'text': 'a...","['red', 'man', 'white', 'woman', 'table', 'cou...",['Red'],"[{'confidence': 0.4687312594679555, 'text': 'a...",VizWiz_train_000000000019.jpg,Can I ask about anything or any information wi...,yes/no,0,...,no,yes,no,no,no,unanswerable,unanswerable,unanswerable,yes,red
7,VizWiz_train_000000000020.jpg,What is this game?,"[{'confidence': 0.5852738219692016, 'text': 'a...","['indoor', 'sitting', 'photo', 'computer', 'ta...","['White', 'Grey']","[{'confidence': 0.5852738219692016, 'text': 'a...",VizWiz_train_000000000020.jpg,What is this game?,other,1,...,grand theft auto,grand theft auto vice city,grand theft auto vice city,grand theft auto,grand theft auto,grand theft auto,grand theft auto vice city,grand theft auto vice city,grand theft auto vice city,grand theft auto vice city
8,VizWiz_train_000000000022.jpg,What is in this can?,[],"['indoor', 'kitchen', 'food', 'sitting', 'tabl...","['Brown', 'Black']",[],VizWiz_train_000000000022.jpg,What is in this can?,other,1,...,light red kidney beans,light red kidney beans,kidney beans,kidney beans,red kidney beans,red kidney beans,light red kidney beans,light red kidney beans,kidney beans,light red kidney beans
9,VizWiz_train_000000000025.jpg,What is this a picture of?,"[{'confidence': 0.6310461589130699, 'text': 'a...","['indoor', 'dog', 'sitting', 'looking', 'cat',...","['Grey', 'Black']","[{'confidence': 0.6310461589130699, 'text': 'a...",VizWiz_train_000000000025.jpg,What 

In [6]:
#evaluate
n 			  = 2
accuracy     = {}
evalQA       = {}
evalAnsType  = {}
contractions = {"aint": "ain't", "arent": "aren't", "cant": "can't", "couldve": "could've", "couldnt": "couldn't", \
							 "couldn'tve": "couldn't've", "couldnt've": "couldn't've", "didnt": "didn't", "doesnt": "doesn't", "dont": "don't", "hadnt": "hadn't", \
							 "hadnt've": "hadn't've", "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent": "haven't", "hed": "he'd", "hed've": "he'd've", \
							 "he'dve": "he'd've", "hes": "he's", "howd": "how'd", "howll": "how'll", "hows": "how's", "Id've": "I'd've", "I'dve": "I'd've", \
							 "Im": "I'm", "Ive": "I've", "isnt": "isn't", "itd": "it'd", "itd've": "it'd've", "it'dve": "it'd've", "itll": "it'll", "let's": "let's", \
							 "maam": "ma'am", "mightnt": "mightn't", "mightnt've": "mightn't've", "mightn'tve": "mightn't've", "mightve": "might've", \
							 "mustnt": "mustn't", "mustve": "must've", "neednt": "needn't", "notve": "not've", "oclock": "o'clock", "oughtnt": "oughtn't", \
							 "ow's'at": "'ow's'at", "'ows'at": "'ow's'at", "'ow'sat": "'ow's'at", "shant": "shan't", "shed've": "she'd've", "she'dve": "she'd've", \
							 "she's": "she's", "shouldve": "should've", "shouldnt": "shouldn't", "shouldnt've": "shouldn't've", "shouldn'tve": "shouldn't've", \
							 "somebody'd": "somebodyd", "somebodyd've": "somebody'd've", "somebody'dve": "somebody'd've", "somebodyll": "somebody'll", \
							 "somebodys": "somebody's", "someoned": "someone'd", "someoned've": "someone'd've", "someone'dve": "someone'd've", \
							 "someonell": "someone'll", "someones": "someone's", "somethingd": "something'd", "somethingd've": "something'd've", \
							 "something'dve": "something'd've", "somethingll": "something'll", "thats": "that's", "thered": "there'd", "thered've": "there'd've", \
							 "there'dve": "there'd've", "therere": "there're", "theres": "there's", "theyd": "they'd", "theyd've": "they'd've", \
							 "they'dve": "they'd've", "theyll": "they'll", "theyre": "they're", "theyve": "they've", "twas": "'twas", "wasnt": "wasn't", \
							 "wed've": "we'd've", "we'dve": "we'd've", "weve": "we've", "werent": "weren't", "whatll": "what'll", "whatre": "what're", \
							 "whats": "what's", "whatve": "what've", "whens": "when's", "whered": "where'd", "wheres": "where's", "whereve": "where've", \
							 "whod": "who'd", "whod've": "who'd've", "who'dve": "who'd've", "wholl": "who'll", "whos": "who's", "whove": "who've", "whyll": "why'll", \
							 "whyre": "why're", "whys": "why's", "wont": "won't", "wouldve": "would've", "wouldnt": "wouldn't", "wouldnt've": "wouldn't've", \
							 "wouldn'tve": "wouldn't've", "yall": "y'all", "yall'll": "y'all'll", "y'allll": "y'all'll", "yall'd've": "y'all'd've", \
							 "y'alld've": "y'all'd've", "y'all'dve": "y'all'd've", "youd": "you'd", "youd've": "you'd've", "you'dve": "you'd've", \
							 "youll": "you'll", "youre": "you're", "youve": "you've"}
manualMap    = { 'none': '0',
							  'zero': '0',
							  'one': '1',
							  'two': '2',
							  'three': '3',
							  'four': '4',
							  'five': '5',
							  'six': '6',
							  'seven': '7',
							  'eight': '8',
							  'nine': '9',
							  'ten': '10'
							}
articles     = ['a',
							 'an',
							 'the'
							]

 

periodStrip  = re.compile("(?!<=\d)(\.)(?!\d)")
commaStrip   = re.compile("(\d)(\,)(\d)")
punct        = [';', r"/", '[', ']', '"', '{', '}',
				'(', ')', '=', '+', '\\', '_', '-',
				'>', '<', '@', '`', ',', '?', '!']

def processPunctuation(inText):
		outText = inText
		for p in punct:
			if (p + ' ' in inText or ' ' + p in inText) or (re.search(commaStrip, inText) != None):
				outText = outText.replace(p, '')
			else:
				outText = outText.replace(p, ' ')	
		outText = periodStrip.sub("",
									 outText,
									 re.UNICODE)
		return outText
	
def processDigitArticle(inText):
		outText = []
		tempText = inText.lower().split()
		for word in tempText:
			word = manualMap.setdefault(word, word)
			if word not in articles:
				outText.append(word)
			else:
				pass
		for wordId, word in enumerate(outText):
			if word in contractions: 
				outText[wordId] = contractions[word]
		outText = ' '.join(outText)
		return outText

    
def setAccuracy(accQA, accAnsType):
		accuracy['overall']         = round(100*float(sum(accQA))/len(accQA), n)
		accuracy['perAnswerType']   = {ansType:  round(100*float(sum(accAnsType[ansType]))/len(accAnsType[ansType]), n) for ansType in accAnsType}
			
def setEvalQA(quesId, acc):
		evalQA[quesId] = round(100*acc,n)


def setEvalAnsType(quesId, ansType, acc):
		if ansType not in evalAnsType:
			evalAnsType[ansType] = {}
		evalAnsType[ansType][quesId] = round(100*acc, n)



In [7]:
accQA       = []
accAnsType  = {}
for i in range(0,len(colorResult)):
    
    resAns      = colorResult.loc[i]['answer']
    
    resAns      = resAns.replace('\n', ' ')
    resAns      = resAns.replace('\t', ' ')
    resAns      = resAns.strip()
    resAns      = processPunctuation(resAns)
    resAns      = processDigitArticle(resAns)
    print(resAns)
    gtAcc  = []
    gtAnswers = [str(colorResult.loc[i]['ANS1']),
                 str(colorResult.loc[i]['ANS2']),
                 str(colorResult.loc[i]['ANS3']),
                 str(colorResult.loc[i]['ANS4']),
                 str(colorResult.loc[i]['ANS5']),
                 str(colorResult.loc[i]['ANS6']),
                 str(colorResult.loc[i]['ANS7']),
                 str(colorResult.loc[i]['ANS8']),
                 str(colorResult.loc[i]['ANS9']),
                 str(colorResult.loc[i]['ANS10'])]
    print(gtAnswers)
    if len(set(gtAnswers)) > 1: 
        for ansDic in gtAnswers:
            ansDic = processPunctuation(ansDic)
    for gtAnsDatum in gtAnswers:
        otherGTAns = [item for item in gtAnswers if item!=gtAnsDatum]
        matchingAns=[]
        for item in otherGTAns:
            if item.isdigit():
                item=' '+item+' '
            if item=="no" or item=="yes":
                item=' '+item+' '
            if item in resAns:
                matchingAns.append(item)
        acc = min(1, float(len(matchingAns))/3)
        gtAcc.append(acc)

    ansType     = colorResult.loc[i]['ANS_TYP']

    avgGTAcc = float(sum(gtAcc))/len(gtAcc)
    print(avgGTAcc)
    accQA.append(avgGTAcc)

    if ansType not in accAnsType:
        accAnsType[ansType] = []

    accAnsType[ansType].append(avgGTAcc)
    setEvalQA(colorResult.loc[i]['qid'], avgGTAcc)
    setEvalAnsType(colorResult.loc[i]['qid'], ansType, avgGTAcc)


setAccuracy(accQA, accAnsType)
print("Done computing accuracy")
	
	

'confidence': 0.3326841203874476 'text': 'a hand holding cell phone'
['monitor cleaning kit', 'monitor cleaning kit', 'cleaning kit', 'monitor cleaning kit', 'monitor cleaning kit', 'monitor cleaner', 'monitor cleaning kit', 'monitor cleaning kit', 'monitor cleaning kit', 'monitor cleaning kit']
0.0

['orange', 'orange', 'orange', 'pink', 'orange', 'orange', 'pink', 'orange', 'orange', 'orange']
0.0
'confidence': 0.9337091944456669 'text': 'a hand holding cellphone'
['android phone', 'cell phone', 'cell phone', 'cellphone', 'samsung phone', 'unanswerable', 'phone', 'smart phone', 'white smartphone', 'phone']
0.8333333333333333
'confidence': 0.910822291974947 'text': 'a hand holding remote control'
['remote', 'remote control', 'jvc dvd vcr remote', 'remote', 'remote', 'remote control', 'remote', 'remote', 'tv remote', 'remote']
0.8
'confidence': 0.5964878314966942 'text': 'a person lying on bed'
['soda', 'dr pepper', 'dr pepper', 'soda', 'dr pepper', 'dr pepper', 'energy drink', 'dr pep

'confidence': 0.5410948670467779 'text': 'a close up of white wall'
['erf', 'not car', 'black', 'grey', 'grey pants', 'unanswerable', 'pocket', 'grey', 'unanswerable', 'unanswerable']
0.0
'confidence': 0.79012142881414 'text': 'a close up of person'
['white black speckles', 'white', 'white black', 'black', 'black white', 'black white', 'white black', 'black white', 'black white', 'hair']
0.0
'confidence': 0.42568048586615403 'text': 'a close up of black bag'
['grey', 'grey', 'grey', 'grey', 'dark grey', 'black', 'blue', 'grey', 'grey', 'grey']
0.3
'confidence': 0.731551628002587 'text': 'a close up of plate on table'
['blue', 'blue white', 'blue', 'blue', 'blue', 'blue', 'blue', 'mostly blue', 'blue red white', 'blue']
0.0
'confidence': 0.9256208639569562 'text': 'a close up of animal'
['tan white', 'beige', 'unanswerable', 'dog', 'brown grey', 'tan', 'grey', 'tabby', 'cat grey', 'brown']
0.0
'confidence': 0.8042057291486933 'text': 'a pair of scissors'
['scissors', 'scissors', 'scisso

0.0

['black', 'unsuitable', 'black', 'black', 'grey', 'black', 'black', 'black', 'black', 'black']
0.0
'confidence': 0.6872243617463019 'text': 'a red and white shower curtain'
['unanswerable', 'table', 'unanswerable', 'table', 'wood', 'reddish brown wood', 'wallpaper', 'table', 'unanswerable', 'unanswerable']
0.0

['red', 'red', 'red', 'red', 'red', 'red', 'red', 'red', 'red', 'red']
0.0
'confidence': 0.8746874035960136 'text': 'a large brown dog lying on floor'
['golden retriever', 'labrador retriever', 'yellow lab', 'labrador', 'old', 'golden retriever', 'lab', 'golden retriever', 'labrador', 'golden retriever mix']
0.0
'confidence': 0.6584184154599048 'text': 'a person in dark cloudy sky'
['partly cloudy', 'cloudy', 'grey', 'some clouds', 'partly cloudy', 'cloudy', 'cloudy', 'cloudy', 'stormy', 'cloudy']
0.5
'confidence': 0.8443802867592356 'text': 'a group of clouds in sky'
['white fluffy cotton', 'fluffy bright white grey spots', 'white fluffy', 'big puffy light to dark grey', '

['black', 'black', 'black', 'black', 'black', 'green', 'black', 'silver', 'black', 'black']
0.0
'confidence': 0.6906369381192217 'text': 'a close up of box'
['diced tomatoes', 'canned tomatoes', 'diced tomatoes', 'diced tomatoes', 'can tomatos', 'diced tomatoes', 'food', 'canned tomatoes', 'diced tomatoes can', 'great value diced tomatoes']
0.0
'confidence': 0.7514506958768858 'text': 'a close up of box'
['black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black']
0.0
'confidence': 0.36603227191851945 'text': 'a person sitting on bed'
['black gold', 'yellow', 'yellow', 'yellow black stripe', 'yellow black', 'black yellow', 'yellow', 'yellow black trim zippers', 'yellow', 'yellow']
0.0
'confidence': 0.4855099065618867 'text': 'a bottle next to cup of coffee'
['soda', 'oversize pepsi can', 'pepsi max', 'pepsi max', 'pepsi max soda', 'pepsi max', 'pepsi max', 'pop', 'can pepsi max', 'pepsi max']
0.0

['campbells chunky', 'chunky soup', 'unsuitable', 'chunky s

['light pink', 'white', 'white', 'light pink', 'white light pink', 'pale pink', 'white', 'white', 'white', 'white']
0.0

['alcohol', 'liquor bottle', 'whisky', 'alcohol', 'alcohol', 'alcohol', 'makers mark whiskey', 'glass liquor bottle', 'makers mark', 'glass']
0.0
'confidence': 0.6288910626871589 'text': 'a close up of red wall'
['red', 'red', 'red', 'red', 'red', 'red', 'red', 'red', 'red', 'red']
0.0
'confidence': 0.8386746235333852 'text': 'a close up of horse'
['grey', 'grey', 'grey', 'grey', 'grey', 'tan', 'grey', 'grey', 'grey', 'grey']
0.0
'confidence': 0.3455863757288524 'text': 'a person that is eating some food'
['white', 'white', 'yellow cream', 'rose', 'light pink', 'creamy yellow', 'white', 'white', 'white', 'white']
0.0
'confidence': 0.5646297627348984 'text': 'a man looking at camera'
['grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'unanswerable']
0.0
'confidence': 0.8904845155991645 'text': 'a yellow flower in tree'
['yellow flowered', 'unansw

...

['very clean', 'unanswerable', 'very', 'clean', 'very clean', 'very clean', 'very', 'very', 'very', 'very clean']
0.0

['stapler', 'stapler', 'stapler', 'stapler', 'stapler', 'stapler', 'stapler', 'stapler', 'stapler', 'stapler']
0.0
'confidence': 0.276756500681135 'text': 'a pair of red shoes'
['red', 'silver', 'red', 'red', 'red', 'pink', 'red', 'red', 'red', 'silver']
0.3

['granola bar', 'nature valley granola bar', 'granola bar', 'nature valley crunchy oats n honey bar', 'nature valley crunchy oats n honey flavor granola bar', 'granola bar', 'nature valley oats n honey bar', 'granola bar', 'nature valley crunchy oats n honey bar', 'breakfast bar']
0.0

['too many objects 0 specified', 'yellow', 'light brown', 'yellow', 'yellow green', 'yellow', 'yellow', 'yellow', 'yellow', 'yellow']
0.0
'confidence': 0.4184756803772099 'text': 'a close up of person'
['purple', 'pink', 'purple', 'pink', 'pink', 'pink', 'purple', 'pink', 'pink', 'purple']
0.0

['maybe mushroom', 'mushroom', 'mushro

'confidence': 0.6269744176113704 'text': 'a close up of zebra with blanket'
['brown white green', 'white green brown', 'green brown white', 'green white tan', 'white green yellow brown stripes', 'white green tan brown stripes', 'white green brown tan striped', 'brown yellow white green', 'white cloth green brown stripes', 'green white brown tan stripes']
0.0
'confidence': 0.8831602166445479 'text': 'a bedroom with large bed in room'
['purple wite green', 'light beige purple green floral pattern', 'cream floral print', 'purple blue flowers greenery on beige background', 'white green purple', 'white green blue purple', 'pink green ivory', 'purple', 'white vailoet', 'beige purple blue green']
0.0
'confidence': 0.32643213937418747 'text': 'a screen shot of glass shower door'
['no', 'no', 'no', 'no', 'no', 'no', 'no', 'maybe', 'no', 'no']
0.0
'confidence': 0.8426508029670282 'text': 'a can of food'
['canned chili', 'can chili no beans', 'chili', 'chili', 'chili no beans', 'chili no beans', 

['batteries', '4 triple batteries', 'pack batteries', 'triple batteries', 'duracell procell triple batteries', 'triple batteries', 'aaa batteries', 'batteries', 'batteries', 'batteries']
0.0
'confidence': 0.9232237370281058 'text': 'a hand holding cellphone'
['mobile phone', 'smartphone', 'cell phone', 'phone', 'cell phone asking for location', 'phone', 't mobile cellphone', 'phone', 'unsuitable', 'cell phone']
0.7

['brown', 'brown', 'brown', 'stripped light dark brown', 'browns', 'brown', 'brown', 'brown', 'gold', 'brown']
0.0
'confidence': 0.574928860815446 'text': 'a close up of person sleeping on blanket'
['white blue', 'white', 'grey', 'white', 'unsuitable', 'light blue', 'white', 'pale blue', 'light', 'light blue denim']
0.0
'confidence': 0.8625511367454125 'text': 'a close up of bottle'
['dr pepper', 'this dr pepper soda', 'unanswerable', 'dr pepper', 'dr pepper', 'dr pepper', 'dr pepper', 'dr pepper', 'dr pepper', 'dr pepper']
0.0
'confidence': 0.5303989231105992 'text': 'a cl

['no', 'no', 'no', 'no', 'no', 'no', 'no', 'unanswerable', 'no', 'unsuitable']
0.0
'confidence': 0.8985700853727665 'text': 'a laptop computer sitting on top of keyboard'
['laptop', 'laptop computer', 'key pad computer screen', 'laptop', 'laptop', 'keyboard', 'laptop computer', 'laptop', 'computer monitor keyboard', 'laptop']
1.0
'confidence': 0.7075917007367064 'text': 'a laptop computer sitting on top of table'
['iphone', 'iphone', 'cell phone', 'cell phone', 'i phone', 'iphone', 'cell phone', 'smart phone', 'mobile phone', 'cell phone']
0.0

['toby mcguire', 'tobey maguire', 'tobey maguire', 'tobey maguire', 'toby mcguire', 'unsuitable', 'tobey maguire', 'spider man', 'toby mcguire', 'spider man']
0.0

['lizards orange left green right', 'green orange', 'left orange right green toy lizards', 'orange green geckos', 'green orange lizards', 'green orange', 'green orange toy lizards', 'green orange lizards', 'orange lizard green lizard', 'toy lizards 1 orange 1 green']
0.0
'confidence':

'confidence': 0.37643547004009115 'text': 'a person sleeping on blanket'
['unanswerable', 'white', 'white', 'white', 'white', 'unanswerable', 'white', 'green', 'tan', 'white']
0.0
'confidence': 0.3240695814475038 'text': 'a red white and blue bowl'
['soft cream cheese', 'soft cream cheese', 'soft cream cheese', 'soft cream cheese', 'soft cream cheese', 'kroger soft cream cheese', 'kroger soft cream cheese', 'soft cream cheese', 'soft cream cheese', 'soft cream cheese']
0.0
'confidence': 0.8182815763962562 'text': 'a close up of sign'
['chocolate cheerios', 'cheerios', 'chocolate cheerios', 'cheerios', 'chocolate cheerios', 'chocolate cheerios', 'chocolate cheerios', 'chocolate cheerios', 'chocolate cheerios', 'chocolate cheerios']
0.0
'confidence': 0.701229862987027 'text': 'a person holding cell phone'
['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'qrgrg']
0.0
'confidence': 0.678683704647425 'text': 'a can of food'
['soda', 'soda', 'soda', 'soda', 'soda', 'coca cola on left s

In [8]:
setAccuracy(accQA, accAnsType)
print("Done computing accuracy")

Done computing accuracy


In [9]:
print(accQA)

[0.0, 0.0, 0.8333333333333333, 0.8, 0.0, 0.1, 0.3, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.3, 0.0, 0.0, 0.5333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5333333333333333, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.1, 0.5, 0.0, 0.0, 0.3, 0.0, 0.3, 0.0, 0.3, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.5333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.3, 0.0, 0.0, 0.5333333333333333, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.1, 0.0, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8333333333333334, 0.0, 0.7,

In [10]:
print(accAnsType)

{'yes/no': [0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [11]:
print(accuracy['overall'])         
print(accuracy['perAnswerType'])  

6.89
{'yes/no': 0.78, 'other': 7.63, 'number': 2.61, 'unanswerable': 4.52}
